In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
import re

import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim

import models as md
from sklearn.linear_model import LogisticRegression


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

//anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
members=pd.read_csv('filtered_members.csv')
groups=pd.read_csv('filtered_groups.csv')

//anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,100,101,102,103,104,105,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,152) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (385,395,450,452,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539

In [5]:
group_titles=pd.read_csv('titles_groups.csv')

//anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (386,396,451,453,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672

In [6]:
# stop_words=set(stopwords.words("english"))
# lemmatizer = WordNetLemmatizer() 

In [7]:
def cleaning_bow_for_training(data):
    
    new_data=data.dropna(how='all', axis=1)
    bow=data.iloc[:,2:].values.tolist()
    
    cleaned_data = [
    [w for w in l if isinstance(w, str)] 
    for l in bow
    ]
    return cleaned_data

In [8]:
members_bow=cleaning_bow_for_training(members)

In [9]:
groups_bow=cleaning_bow_for_training(groups)

In [10]:
training_bow=members_bow+groups_bow

In [11]:
model = gensim.models.Word2Vec(training_bow, min_count=1, sg=1)
model.train(training_bow, total_examples=model.corpus_count, epochs=model.epochs)
model.wv

In [12]:

def tags_from_lists(bow):
    tags_lists=[]
    for lst in bow:
        while True:
            if len(lst) == 0:
                break
            try:
                tags=model.wv.most_similar(positive=lst, topn=20)
                tags_lists.append(tags)
            except KeyError as e:
                search = re.search("'(\w+)'", e.args[0])
                word = search.group(1)
                print(word)
                lst.remove(word)
            else:
                break
    return tags_lists

In [13]:
tags_groups=tags_from_lists(groups_bow)

In [19]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=set(stopwords.words("english"))
def transforming_in_bow(text):
    
    sentence = text.translate(str.maketrans('', '',string.punctuation)).split(' ')
    bow = []
    for word in sentence:
        lowcase_text_word=word.lower()
        lemmatized_word=lemmatizer.lemmatize(lowcase_text_word)
        if lemmatized_word not in stop_words:
            bow.append(lemmatized_word)
        
    return bow

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/flatironschool/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
bow = transforming_in_bow('I am an architect who likes to explore cities and analyse the data about them.')

In [21]:
bow

['architect', 'like', 'explore', 'city', 'analyse', 'data']

In [22]:
def tags_from_text(bow):
    while True:
        if len(lst) == 0:
            break
        try:
            tags=model.wv.most_similar(positive=bow, topn=20)
        except KeyError as e:
            search = re.search("'(\w+)'", e.args[0])
            word = search.group(1)
            print(word)
            bow.remove(word)
        else:
            break
    return tags

In [23]:
tags_text=tags_from_text(bow)

NameError: name 'lst' is not defined

In [ ]:
tags_text

In [24]:
import statistics
def relevance(member_bow, groups_bow):
    distance_lists_max=[]
    for group in groups_bow:
        group_id=[]
        for word_g in group:
            for word_m in member_bow:
                distance=model.wv.distance(word_m, word_g)
                proportion=1-distance
                group_id.append(proportion)
        distance_lists_max.append(statistics.mean(group_id))
    return distance_lists_max

In [25]:
tags_relevance=relevance(bow, groups_bow)

In [26]:
tags_relevance

[0.4148430546367663,
 0.41814063122819844,
 0.4068466676664232,
 0.4009041379201837,
 0.3931791218058815,
 0.39570429581238664,
 0.4085396249817975,
 0.3739060924752485,
 0.41448741293067787,
 0.4053451374654063,
 0.41152978617118774,
 0.41510503729405207,
 0.41831475258459044,
 0.3991134448661576,
 0.38444819673960734,
 0.3990412838193761,
 0.3978123239378641,
 0.4399526326310537,
 0.39327108526933807,
 0.429690006358272,
 0.39716439656754676,
 0.38817313693962197,
 0.4333337321983243,
 0.427446232833318,
 0.41282917745555653,
 0.4292450974758774,
 0.43316158005117905,
 0.3802933736735012,
 0.4021594404864806,
 0.4219056462127057,
 0.38796259192929416,
 0.3949893000919317,
 0.39720754114484913,
 0.3929790182447833,
 0.4009824920366133,
 0.4278885831702221,
 0.40540515077013617,
 0.39456900545688217,
 0.4307571742748725,
 0.39232729299322705,
 0.40593678243752856,
 0.424475292578198,
 0.40895885957405953,
 0.38390878317971605,
 0.4065293985468546,
 0.4026749423520981,
 0.40342462085652

In [44]:
def get_the_recommended_titles(relevance, titles_data):
    distance_list_titles=pd.DataFrame(relevance, columns=['member'])
    titles_values=pd.merge(left=titles_data[['group_name']], right=distance_list_titles, left_index=True, right_index=True)
    recommended_gr=titles_values.loc[titles_values['member']>=0.50]
    return recommended_gr[['group_name']]

In [39]:
get_the_recommended_titles(tags_relevance, group_titles)

,group_name
5478,SF Bay Area Ukrainians and Friends!
5987,SF Cloud Networking
9111,New York Cloud Gazing
9687,Callback Night
9881,New York City SAS / Data Analytics
13829,New York CMS (Content Management Systems) Meetup
13921,Traveling To New Adventures


In [45]:
def input_text_recommendation(text):
    bow = transforming_in_bow(text)
#     tags_text=tags_from_text(bow)
    tags_relevance=relevance(bow, groups_bow)
    recommended=get_the_recommended_titles(tags_relevance, group_titles)
    return recommended

In [46]:
input_text_recommendation('I like football')

,group_name
4199,San Francisco Giant Fans
5478,SF Bay Area Ukrainians and Friends!
8179,TailDating New York
8600,Bay Area Bollywood Events
8937,PlayBallChicago
14657,New England Patriots Game Meetup of Manhattan


In [47]:
input_text_recommendation('I enjoy good, complex science fiction, especially space opera')

,group_name


In [48]:
input_text_recommendation('I enjoy science fiction, especially space opera')

,group_name


In [49]:
input_text_recommendation('I have been in africa for a year, saving the cangurous from starving, and now i am back to find more people who wants to be volunteer in order to save the nature')

KeyError: "word 'cangurous' not in vocabulary"

In [50]:
input_text_recommendation('I like cooking')

,group_name
322,The Chicago Yoga Meetup Group
457,The San Francisco Brunch Club
530,Drinking With New Friends
677,I LOVE San Francisco!!!
828,The San Francisco Metal Meetup Group
1032,KARAOKE DANCE PARTY
2252,"! BOOZY BOOKWORMS, MOVIES, FRIENDS, & SOCIAL E..."
2420,Fantastic 50s Plus
3567,Absinthe reborn
3674,Eaters Drinkers Meetup


In [51]:
input_text_recommendation('I would like to know more about robots')

,group_name
13630,NYC Spankos


In [52]:
input_text_recommendation('I am learning now python. It will be very useful for me to find some people from IT')

,group_name
138,The San Francisco Ruby Meetup Group
2041,Bot UX
5478,SF Bay Area Ukrainians and Friends!
8422,Full Stack Coding
9687,Callback Night
12709,Chicago ReasonML
13921,Traveling To New Adventures
14988,CSS Animation
15743,HSV Herpes Dating meetup
16153,San Francisco ICO's Meetup


In [53]:
input_text_recommendation('I want to visit more art expo')

,group_name
